# Data Cleaning for OpenFoodFacts

## Dataset Description

Dataset Description...

In [1]:
# Import libraries

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import csv

#import re
import regex as re

import os
from pathlib import Path

In [2]:
# Parameters

path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
openfoodfactsUrl = path + '/data/en.openfoodfacts.org.products.2.csv'
foodtableUrl = path + '/data/data_2.tsv'

ingredients_tx = path + '/data/ingredients.txt'
packaging_materials_tx = path + '/data/packaging_materials.txt'
packaging_shapes_tx = path + '/data/packaging_shapes.txt'

savePath =  path + '/data/foodDB/'

## Helper Functions

Helper functions...

In [ ]:
# Plotting the dataset

def plotMissingValues(dataframe, width, high, character=False):
    plt.figure(figsize=(width, high)) 
    if character==False:
        ax = dataframe.isnull().sum().plot.barh(color='skyblue') 
    else:
        occurrences = (dataframe == character).sum().sort_values(ascending=True)
        ax = occurrences.plot.barh(color='skyblue') 


    # Customizing the plot

    plt.title('Missing Values per Column', fontsize=16)
    plt.xlabel('Number of Missing Values', fontsize=12)
    plt.ylabel('Columns', fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.7) 
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)


    # Display the values on the bars
    
    if character==False:
        for i, v in enumerate(dataframe.isnull().sum()):
            ax.text(v + 500, i, str(v), color='gray', va='center', fontsize=8)
    else:
        for i, v in enumerate(occurrences):
            ax.text(v + 500, i, str(v), color='gray', va='center', fontsize=8)

    plt.tight_layout()
    plt.show()

## Import Data (Chunk)

Load the data (one chunk) from a .tsv file

In [ ]:
# Load the data
chunk_size = 100000
chunks_df = pd.read_csv(openfoodfactsUrl, sep="\t", chunksize=chunk_size)

for chunk_df in chunks_df:
    print(chunk_df.shape)
    break

## Data Exploration (Chunk)

Data Exploration...

In [ ]:
# View columns name

print(chunk_df.columns.values)

chunk_df.shape

In [ ]:
# Plotting the dataset
    
plotMissingValues(chunk_df, 7, 30)

## Import Data

Load the data from a .tsv file

In [ ]:
# Create the filter
# $ = keep as is | - = don't keep | ? - keep as optional | t = translate

df_filter = ["code", #$$$$$$
             "product_name", #$$$$$$
             "packaging_tags", #?t?t?t (enumerate)
             "brands", #??????
             "brands_tags", #??????
             "labels_en", #t?t?t? (da processare)
             "countries_en", #$$$$$$ (importare il dataset poi - o countries-tags)
             "ingredients_tags", #$t$t$t (tassonomia - o ingredients_text o ingredients_analysis_tags)
             "serving_size", #??????
             "additives_n", #------ Usato solo per processare additives_tags
             "additives_tags", #?????? (importare il dataset poi)
             "nutriscore_score", #??????
             "nutriscore_grade", #??????
             "nova_group", #??????
             "food_groups", #??????
             "food_groups_tags", #?????? (o food_groups_en)
             "brand_owner", #??????
             "ecoscore_grade", #??????
             #"main_category_en", #------ (sostituito da food_groups_en)
             "energy-kcal_100g", #$$$$$$
             "energy_100g", #$$$$$$
             "fat_100g", #$$$$$$$
             "saturated-fat_100g", #??????
             "trans-fat_100g", #??????
             "cholesterol_100g", #??????
             "carbohydrates_100g", #$$$$$$
             "sugars_100g", #??????
             "fiber_100g", #??????
             "proteins_100g", #$$$$$$
             "salt_100g", #??????
             "sodium_100g", #??????
             "vitamin-a_100g", #??????
             "vitamin-c_100g", #??????
             "calcium_100g", #??????
             "iron_100g"] #??????

In [ ]:
filter_df = pd.read_csv(openfoodfactsUrl, sep="\t", usecols=df_filter)

## Data Exploration

Data Exploration...

In [ ]:
# View columns name

print(filter_df.columns.values)

filter_df.shape

In [ ]:
# Plotting the dataset
    
plotMissingValues(filter_df, 7, 6)

## Data Cleaning

Data Cleaning...

In [ ]:
# Replace null values in additives_tags with empty string if additives_n is 0

def replace_null(row):
    if row['additives_n'] == 0 and pd.isnull(row['additives_tags']):
        return ""
    else:
        return row['additives_tags']
    
def replace_null_bycoll(row, col):
    if pd.isnull(row[col]):
        return ""
    else:
        return row[col]
    

filter_df['additives_tags'] = filter_df.apply(replace_null, axis=1)

filter_df["packaging_tags"] = filter_df.apply(replace_null_bycoll, args=('packaging_tags',), axis=1)
filter_df["brands"] = filter_df.apply(replace_null_bycoll, args=('brands',), axis=1)
filter_df["brands_tags"] = filter_df.apply(replace_null_bycoll, args=('brands_tags',), axis=1)
filter_df["labels_en"] = filter_df.apply(replace_null_bycoll, args=('labels_en',), axis=1)
filter_df["serving_size"] = filter_df.apply(replace_null_bycoll, args=('serving_size',), axis=1)
filter_df["nutriscore_score"] = filter_df.apply(replace_null_bycoll, args=('nutriscore_score',), axis=1)
filter_df["nutriscore_grade"] = filter_df.apply(replace_null_bycoll, args=('nutriscore_grade',), axis=1)
filter_df["nova_group"] = filter_df.apply(replace_null_bycoll, args=('nova_group',), axis=1)
filter_df["food_groups"] = filter_df.apply(replace_null_bycoll, args=('food_groups',), axis=1)
filter_df["food_groups_tags"] = filter_df.apply(replace_null_bycoll, args=('food_groups_tags',), axis=1)
#filter_df["food_groups_en"] = filter_df.apply(replace_null_bycoll, args=('food_groups_en',), axis=1)
filter_df["brand_owner"] = filter_df.apply(replace_null_bycoll, args=('brand_owner',), axis=1)
filter_df["ecoscore_grade"] = filter_df.apply(replace_null_bycoll, args=('ecoscore_grade',), axis=1)
#filter_df["main_category_en"] = filter_df.apply(replace_null_bycoll, args=('main_category_en',), axis=1)
filter_df["saturated-fat_100g"] = filter_df.apply(replace_null_bycoll, args=('saturated-fat_100g',), axis=1)
filter_df["trans-fat_100g"] = filter_df.apply(replace_null_bycoll, args=('trans-fat_100g',), axis=1)
filter_df["cholesterol_100g"] = filter_df.apply(replace_null_bycoll, args=('cholesterol_100g',), axis=1)
filter_df["sugars_100g"] = filter_df.apply(replace_null_bycoll, args=('sugars_100g',), axis=1)
filter_df["fiber_100g"] = filter_df.apply(replace_null_bycoll, args=('fiber_100g',), axis=1)
filter_df["salt_100g"] = filter_df.apply(replace_null_bycoll, args=('salt_100g',), axis=1)
filter_df["sodium_100g"] = filter_df.apply(replace_null_bycoll, args=('sodium_100g',), axis=1)
filter_df["vitamin-a_100g"] = filter_df.apply(replace_null_bycoll, args=('vitamin-a_100g',), axis=1)
filter_df["vitamin-c_100g"] = filter_df.apply(replace_null_bycoll, args=('vitamin-c_100g',), axis=1)
filter_df["calcium_100g"] = filter_df.apply(replace_null_bycoll, args=('calcium_100g',), axis=1)
filter_df["iron_100g"] = filter_df.apply(replace_null_bycoll, args=('iron_100g',), axis=1)

In [ ]:
# Apply the filter to the Dataframe

food_table = filter_df[df_filter].copy()

food_table["isempty"] = np.where(food_table.isnull().sum(axis=1) >= 1, 1, 0)
percentage = food_table.isempty.value_counts()[1] / food_table.shape[0] * 100
print("Percentage of dropped rows: " + str(percentage))

food_table = food_table[food_table.isempty==0].copy()
food_table.isnull().sum()

food_table.drop("isempty", inplace=True,axis=1)
food_table.dropna(axis = 0, how = "any", inplace=True)

In [ ]:
food_table.shape

In [ ]:
# Plotting the dataset
    
plotMissingValues(food_table, 7, 6, character="")

In [4]:
pd.set_option('display.max_columns', None)
food_table.head()

,code,product_name,packaging_tags,brands,brands_tags,labels_en,countries_en,ingredients_tags,serving_size,additives_n,additives_tags,nutriscore_score,nutriscore_grade,nova_group,food_groups,food_groups_tags,brand_owner,ecoscore_grade,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g
0,1199,Solène céréales poulet,NaN,CROUS,crous,NaN,France,"en:antioxidant,en:colour,en:tomato,en:vegetabl...",NaN,5.0,"en:e150,en:e160a,en:e202,en:e316,en:e466",NaN,NaN,4.0,NaN,NaN,NaN,NaN,219.0,916.0,5.9,0.5,NaN,NaN,30.3,1.7,2.8,9.7,0.464,0.1856,NaN,NaN,NaN,NaN
1,1663,Crème dessert chocolat,NaN,Ferme De La Frémondière,ferme-de-la-fremondiere,NaN,France,"en:whole-milk,en:dairy,en:milk,en:sugar,en:add...",NaN,1.0,en:e406,NaN,NaN,4.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.000,0.0000,NaN,NaN,NaN,NaN
2,2264,Baguette Poitevin,NaN,Crous resto,crous-resto,NaN,France,"fr:baguette-poite-vin-pain-baguette,en:water,e...",NaN,0.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,207.0,866.0,6.7,3.8,NaN,NaN,27.5,0.6,1.7,8.9,0.400,0.1600,NaN,NaN,NaN,NaN
3,3827,Suedois saumon,NaN,Crous,crous,NaN,France,"fr:paln-suedois,en:water,en:rye-flour,en:flour...",NaN,2.0,"en:e300,en:e503",NaN,NaN,4.0,NaN,NaN,NaN,NaN,172.0,720.0,4.4,1.2,NaN,NaN,23.3,4.6,2.7,8.4,3.580,1.4320,NaN,NaN,NaN,NaN
4,4510,Salade shaker taboulé,NaN,Crous,crous,NaN,France,"fr:taboule,en:vegetable,en:colza-oil,en:oil-an...",NaN,1.0,en:e202,NaN,NaN,4.0,NaN,NaN,NaN,NaN,114.0,477.0,8.1,0.9,NaN,NaN,5.7,1.4,1.7,3.9,0.310,0.1240,NaN,NaN,NaN,NaN


## Save or Load the Dataset

In [ ]:
# Save the new Dataset

food_table.to_csv(path + '/data/data_2.tsv', sep='\t', index=False)

In [3]:
# Import the Dataset

#food_table = pd.read_csv(foodtableUrl, sep="\t", na_values=['unknown'], keep_default_na=False)
food_table = pd.read_csv(foodtableUrl, sep="\t", na_values='unknown')

C:\Users\vitto\AppData\Local\Temp\ipykernel_16408\2897025573.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  food_table = pd.read_csv(foodtableUrl, sep="\t", na_values='unknown')


## Ingredients Counter

In [ ]:
ingredients_map = {}

def cl_ingredients(text, ingredients_map, debug = False):
    

    # Pattern to split the text
    
    # pattern = r"[,.()\[\]{}]"
    pattern = r"[,]"
    split_text = re.split(pattern, text)

    for ingredient in split_text:
        if ingredient in ingredients_map:
            ingredients_map[ingredient] += 1
        else:
            ingredients_map[ingredient] = 1
    
   
    if debug:
        print(text)
#        print(text)
        
    #return ', '.join(result)

#cl_ingredients(food_table.at[32, 'ingredients_tags'], ingredients_map, False)
food_table['ingredients_tags'].apply(cl_ingredients, args=(ingredients_map,))
print(len(ingredients_map))

In [ ]:
# File path to save the CSV data
file_path = path + '/data/ingrediets_2.tsv'

sorted_ingredients = dict(sorted(ingredients_map.items(), key=lambda x: x[1], reverse=True))

# Writing the dictionary to a CSV file with tab-separated values
with open(file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerow(['Ingredient', 'Count'])  # Writing header
    for ingredient, count in sorted_ingredients.items():
        writer.writerow([ingredient, count])

print(f"Dictionary has been written to {file_path}")

## Read Taxonomy

In [ ]:
def split_line(text):
    
    split_text = re.split(':', text)
    
    language_tag = split_text[0]
    ingredients_text = split_text[1]
    
    ingredients = re.split(', ', ingredients_text)
    stripped_ingredients = [ingredient.strip() for ingredient in ingredients]
    
    return language_tag, stripped_ingredients

Ingredients

In [ ]:
# Read the taxonomy line by line and add each element to a dictionary

ingredients_tx_map = {}
master = ''
master_tag = ''
start = True
count_masters = 0


with open(ingredients_tx, 'r', encoding='utf-8') as file:
    
    for line in file:
        
        # Iterate until ingredients
        if start and '# # # # # # # # # # # # # #' not in line:
            continue
        else:
            start = False
            
        # Check if there is a new section
        if line == '\n':
            master = ''
            #print('reset')
            
        # Skip the line if is a comment or if is not a valid line
        if '#' in line or line.count(':') != 1 or '<' in line:
            continue
        
        language_tag, ingredients = split_line(line)
        
        # Add the line to the dictionary
        if language_tag == 'en' or master == '':
            master = ingredients[0]
            master_tag = language_tag
            count_masters += 1
        
        for ingredient in ingredients:
            ingredient = ingredient
            ingredients_tx_map[language_tag+':'+ingredient] = master_tag+':'+master
        
        
    print('# of elements in the dictionary: ' + str(len(ingredients_tx_map)))
    print('# of master elements in the dictionary: ' + str(count_masters) + '\n')
    print(ingredients_tx_map)
        


Packaging

In [ ]:
# Read the taxonomy line by line and add each element to a dictionary

packaging_tx_map = {}
master = ''
master_tag = ''
start = True
count_masters = 0


with open(packaging_materials_tx, 'r', encoding='utf-8') as file:
    
    for line in file:
        
        # Iterate until ingredients (deactivated)
        if start and 'en:Plastic' not in line:
            continue
        else:
            start = False
            
        # Check if there is a new section
        if line == '\n':
            master = ''
            #print('reset')
            
        # Skip the line if is a comment or if is not a valid line
        if '#' in line or line.count(':') != 1 or '<' in line:
            continue
        
        line = line.lower() #IMPORTANT
        language_tag, ingredients = split_line(line)
        
        # Add the line to the dictionary
        if language_tag == 'en' or master == '':
            master = ingredients[0]
            master_tag = language_tag
            count_masters += 1
        
        for ingredient in ingredients:
            ingredient = ingredient
            packaging_tx_map[language_tag+':'+ingredient] = master_tag+':'+master
        
        
    print('# of elements in the dictionary: ' + str(len(packaging_tx_map)))
    print('# of master elements in the dictionary: ' + str(count_masters) + '\n')
        
        
        
master = ''
master_tag = ''
start = True        
        
with open(packaging_shapes_tx, 'r', encoding='utf-8') as file:
    
    for line in file:
        
        # Iterate until ingredients (deactivated)
        if start and 'en:packaging' not in line:
            continue
        else:
            start = False
            
        # Check if there is a new section
        if line == '\n':
            master = ''
            #print('reset')
            
        # Skip the line if is a comment or if is not a valid line
        if '#' in line or line.count(':') != 1 or '<' in line:
            continue
        
        line = line.lower() # IMPORTANT
        language_tag, ingredients = split_line(line)
        
        # Add the line to the dictionary
        if language_tag == 'en' or master == '':
            master = ingredients[0]
            master_tag = language_tag
            count_masters += 1
        
        for ingredient in ingredients:
            ingredient = ingredient
            packaging_tx_map[language_tag+':'+ingredient] = master_tag+':'+master
        
        
    print('# of elements in the dictionary: ' + str(len(packaging_tx_map)))
    print('# of master elements in the dictionary: ' + str(count_masters) + '\n')
    print(packaging_tx_map)


## Data Visualization

Data Visualization...

In [ ]:
# Splits the entries in a DataFrame's target column based on a specified separator, 
# creating new rows for each element resulting from the split.

def splitDataFrameList(df, target_column, separator):
    def splitListToRows(row, row_accumulator, target_column, separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [ ]:
# Print original dataset

countries = df["countries_en"].value_counts()

In [ ]:
# Plotting the horizontal bar chart

ax = countries[:15][::-1].plot.barh(figsize=(8, 4), color='skyblue')
plt.xlabel('Values')
plt.title('Top 15 Countries')


# Displaying the value of each bar on the plot

for i, v in enumerate(countries[:15][::-1]):
    ax.text(v + 500, i, str(v), color='black', va='center', fontsize=10)

plt.show()

In [ ]:
# Print new dataset

food_countries = splitDataFrameList(food_table, "countries_en", ",")
countries = food_countries["countries_en"].value_counts()

In [ ]:
# Plotting the horizontal bar chart

ax = countries[:15][::-1].plot.barh(figsize=(8, 4), color='skyblue')
plt.xlabel('Values')
plt.title('Top 15 Countries')


# Displaying the value of each bar on the plot

for i, v in enumerate(countries[:15][::-1]):
    ax.text(v + 500, i, str(v), color='black', va='center', fontsize=10)

plt.show()

In [ ]:
# Print new dataset

food_countries = splitDataFrameList(food_table, "ingredients_tags", ",")
countries = food_countries["ingredients_tags"].str.split(',').explode().str.strip().value_counts()

In [ ]:
# Plotting the horizontal bar chart

ax = countries[:15][::-1].plot.barh(figsize=(8, 4), color='skyblue')
plt.xlabel('Values')
plt.title('Top 15 Countries')


# Displaying the value of each bar on the plot

for i, v in enumerate(countries[:15][::-1]):
    ax.text(v + 500, i, str(v), color='black', va='center', fontsize=10)

plt.show()

In [ ]:
print(countries)

In [ ]:
countries.to_csv(path + '/data/ingredients.tsv', sep='\t', index=True)

## Populate DB

In [ ]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD

In [ ]:
# Construct the country and the movie ontology namespaces not known by RDFlib
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
FO = Namespace("http://www.graphle.com/foodOntology#")

In [ ]:
def capitalize_first_letters(text):
    return ' '.join(str(text).split('-')).title()

def remove_language_tag(text):
    split_text = re.split(':', str(text))
    
    if len(split_text) > 1:
        return split_text[1]
    print("No language tag in " + str(text))
    return split_text[0]

def create_tag(text):
    lower = str(text).lower().strip()
    cleared = re.sub(r'[^\p{L}\d\s-]', "", lower, flags=re.UNICODE)
    return '-'.join(cleared.split())

def split_tags(text):
    return str(text).split(',')

def filter_tags(text, dictionary):
    result = []
    if pd.isna(text):
        return result
    
    splitted = split_tags(text)
    
    for item in splitted:
        if item in dictionary:
            result.append(dictionary[item])
    return result
    

### Brand

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [ ]:
#iterate over the league dataframe
for index, row in food_table.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the league id as URI
    idU = create_tag(row['brands_tags'])
    
    
    if not pd.isna(idU) and not idU == '':
        Brand = URIRef(FO[idU])
        # Add triples using store's add() method.
        if not (Brand, None, None) in g:   
            g.add((Brand, RDF.type, FO.Brand))
            if not pd.isna(row['brands']):
                g.add((Brand, FO['name'], Literal(capitalize_first_letters(row['brands']), datatype=XSD.string)))  
            else:
                g.add((Brand, FO['name'], Literal(capitalize_first_letters(row['brands_tags']), datatype=XSD.string)))
            if not pd.isna(row['brand_owner']):
                g.add((Brand, FO['owner'], Literal(row['brand_owner'], datatype=XSD.string)))
    
    
    if index % 150000 == 0 :
        print(f"Progress: {index/food_table.shape[0]*100:.2f}%")
        
    
    
    #if index == 100:
    #    break

In [ ]:
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'brand.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

### FoodGroup

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [ ]:
#iterate over the league dataframe
for index, row in food_table.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the league id as URI
    idU = row['food_groups']
    if not pd.isna(idU):
        
        idU = remove_language_tag(row['food_groups'])
        FoodGroup = URIRef(FO[idU])
        # Add triples using store's add() method.
        if not (FoodGroup, None, None) in g:   
            g.add((FoodGroup, RDF.type, FO.FoodGroup))
            g.add((FoodGroup, FO['name'], Literal(capitalize_first_letters(idU), datatype=XSD.string)))  
    
    
    if index % 150000 == 0 :
        print(f"Progress: {index/food_table.shape[0]*100:.2f}%")
        
    # C'E' UN UNKNOWN
    
    
    #if index == 100:
    #    break

In [ ]:
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'foodgroups.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

### Product

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [ ]:
#iterate over the league dataframe
for index, row in food_table.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the league id as URI
    idU = create_tag(row['product_name']) + '-' + str(row['code'])
    
    if idU.startswith('-'):
        continue
    
    Product = URIRef(FO[idU])
    
    # Add triples using store's add() method.
    g.add((Product, RDF.type, FO.Product))
    g.add((Product, FO['name'], Literal(row['product_name'], datatype=XSD.string)))  
    if not pd.isna(row['serving_size']):
        g.add((Product, FO['servingSize'], Literal(row['serving_size'], datatype=XSD.string)))
    if not pd.isna(row['nutriscore_score']):
        g.add((Product, FO['nutriscoreScore'], Literal(row['nutriscore_score'], datatype=XSD.float)))
    if not pd.isna(row['nutriscore_grade']):
        g.add((Product, FO['nutriscoreGrade'], Literal(row['nutriscore_grade'], datatype=XSD.string)))
    if not pd.isna(row['nova_group']):
        g.add((Product, FO['novaGroup'], Literal(row['nova_group'], datatype=XSD.float)))
    if not pd.isna(row['nova_group']):
        g.add((Product, FO['ecoscoreGrade'], Literal(row['ecoscore_grade'], datatype=XSD.string)))   
    g.add((Product, FO['energyKJ'], Literal(row['energy_100g'], datatype=XSD.float)))
    g.add((Product, FO['energyKcal'], Literal(row['energy-kcal_100g'], datatype=XSD.float)))
    g.add((Product, FO['fat'], Literal(row['fat_100g'], datatype=XSD.float)))
    if not pd.isna(row['saturated-fat_100g']):
        g.add((Product, FO['fatSaturated'], Literal(row['saturated-fat_100g'], datatype=XSD.float)))
    if not pd.isna(row['trans-fat_100g']):
        g.add((Product, FO['fatTrans'], Literal(row['trans-fat_100g'], datatype=XSD.float)))
    if not pd.isna(row['cholesterol_100g']):    
        g.add((Product, FO['cholesterol'], Literal(row['cholesterol_100g'], datatype=XSD.float)))
    g.add((Product, FO['carbohydrates'], Literal(row['carbohydrates_100g'], datatype=XSD.float)))
    if not pd.isna(row['sugars_100g']):
        g.add((Product, FO['sugars'], Literal(row['sugars_100g'], datatype=XSD.float)))
    if not pd.isna(row['fiber_100g']):
        g.add((Product, FO['fiber'], Literal(row['fiber_100g'], datatype=XSD.float)))
    g.add((Product, FO['proteins'], Literal(row['proteins_100g'], datatype=XSD.float)))
    if not pd.isna(row['salt_100g']):
        g.add((Product, FO['salt'], Literal(row['salt_100g'], datatype=XSD.float)))
    if not pd.isna(row['sodium_100g']):
        g.add((Product, FO['sodium'], Literal(row['sodium_100g'], datatype=XSD.float)))
    if not pd.isna(row['vitamin-a_100g']):    
        g.add((Product, FO['vitaminA'], Literal(row['vitamin-a_100g'], datatype=XSD.float)))
    if not pd.isna(row['vitamin-c_100g']):
        g.add((Product, FO['vitaminC'], Literal(row['vitamin-c_100g'], datatype=XSD.float)))
    if not pd.isna(row['calcium_100g']):
        g.add((Product, FO['calcium'], Literal(row['calcium_100g'], datatype=XSD.float)))
    if not pd.isna(row['iron_100g']):
        g.add((Product, FO['irom'], Literal(row['iron_100g'], datatype=XSD.float)))
    
    # CI SONO DEGLI UNKNOWN
    
    if index % 25000 == 0 :
        print(f"Progress: {index/food_table.shape[0]*100:.2f}%")
        
    if index == 100000:
        break

In [ ]:
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'product.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))

### Product Join

In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("fo", FO)

In [ ]:
#iterate over the league dataframe
for index, row in food_table.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the league id as URI
    idPr = create_tag(row['product_name']) + '-' + str(row['code'])
    idBr = create_tag(row['brands_tags'])
    idFG = row['food_groups']
    
    if idPr.startswith('-'):
        continue
    Product = URIRef(FO[idPr])
    
    # Join with Brand
    if not pd.isna(idBr) and not idBr == '':
        
        Brand = URIRef(FO[idBr])
        
        g.add((Product, FO['hasBrand'], Brand))

    # Join with FoodGroup
    if not pd.isna(idFG):
        
        idFG = remove_language_tag(row['food_groups'])
        FoodGroup = URIRef(FO[idFG])
        
        g.add((Product, FO['belongsToGroup'], FoodGroup))
        
    # Join with Ingredient
    ing_filtered = filter_tags(row['ingredients_tags'], ingredients_tx_map)
    for ingredient in ing_filtered:
        
        idIn = 'ingredient' + str(index) # PLACEHOLDER INGREDIENT TAG
        Ingredient = URIRef(FO[idIn])
        
        g.add((Product, FO['hasIngredient'], Ingredient))
        
    # Join with Ingredient
    pack_filtered = filter_tags(row['packaging_tags'], packaging_tx_map)
    for package in pack_filtered:
        
        idPa = 'package' + str(index) # PLACEHOLDER INGREDIENT TAG
        Packaging = URIRef(FO[idPa])
        
        g.add((Product, FO['hasPackaging'], Packaging))
    
    
    
    if index % 25000 == 0 :
        print(f"Progress: {index/food_table.shape[0]*100:.2f}%")
        
    if index == 100000: #100000
        break

In [ ]:
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'product_join.ttl', 'w', encoding='utf-8') as file:
    file.write(g.serialize(format='turtle'))